<a href="https://colab.research.google.com/github/Gaurav-Yadav04/Cattle-Breed-Classification/blob/main/breed_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -----------------------------
# Install required libraries (Colab)
# -----------------------------
!pip install tensorflow opencv-python matplotlib

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2

# -----------------------------
# Parameters
# -----------------------------
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
class_names = [ 'Gir',  'Jersey cattle', 'Sahiwal']
NUM_CLASSES = len(class_names)
MAX_EPOCHS = 30

# -----------------------------
# Data Augmentation + Preprocessing
# -----------------------------
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    zoom_range=0.3,
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=[0.6, 1.4],
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/Cattle Breeds",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    "/content/drive/MyDrive/Cattle Breeds",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# -----------------------------
# MobileNetV2 Base Model
# -----------------------------
base_model = MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

# Freeze base initially
base_model.trainable = False

# Hybrid Model
model = models.Sequential([
    layers.Input(shape=(224,224,3)),
    base_model,

    # Custom CNN Layers
    layers.Conv2D(128, (3,3), activation='relu', padding='same'),
    layers.MaxPooling2D(2,2),
    layers.Conv2D(64, (3,3), activation='relu', padding='same'),
    layers.GlobalAveragePooling2D(),

    # Dense Head
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# Loss with label smoothing
loss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=loss,
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# Callbacks
# -----------------------------
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# -----------------------------
# Train base model
# -----------------------------
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=MAX_EPOCHS,
    callbacks=[early_stop, reduce_lr]
)

# -----------------------------
# Fine-tune last 20 layers
# -----------------------------
base_model.trainable = True
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=loss,
    metrics=['accuracy']
)

fine_tune_epochs = 10
total_epochs = MAX_EPOCHS + fine_tune_epochs

history_fine = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=total_epochs,
    initial_epoch=history.epoch[-1]+1,
    callbacks=[early_stop, reduce_lr]
)

# -----------------------------
# Save final model
# -----------------------------
model.save("hybrid_mobilenetv2_model.keras")

# -----------------------------
# Prediction function
# -----------------------------
def preprocess(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, IMAGE_SIZE)
    img = img.astype(np.float32)/255.0
    img = np.expand_dims(img, axis=0)
    return img

def predict(img):
    img_input = preprocess(img)
    preds = model.predict(img_input)[0]
    predicted_index = np.argmax(preds)
    predicted_class = class_names[predicted_index]
    confidence = float(preds[predicted_index])
    return {"class": predicted_class, "confidence": confidence}


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

# -----------------------------
# Get true labels and predictions
# -----------------------------
y_true = []
y_pred = []

for images, labels in val_generator:
    preds = model.predict(images)
    y_true.extend(np.argmax(labels, axis=1))
    y_pred.extend(np.argmax(preds, axis=1))

    if len(y_true) >= val_generator.samples:
        break

y_true = np.array(y_true[:val_generator.samples])
y_pred = np.array(y_pred[:val_generator.samples])

# -----------------------------
# Compute Confusion Matrix
# -----------------------------
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)

# -----------------------------
# Plot Confusion Matrix
# -----------------------------
plt.figure(figsize=(8,8))
disp.plot(cmap=plt.cm.Blues, values_format='d')
plt.title("Confusion Matrix")
plt.show()


In [ ]:
model.evaluate(val_generator)